# Analyze the output of Bayesian Fitting
To run this notebook on your MCMC: <br>
1. Update the idbeg and idend values to match the Run folders containing your simulation <br>
2. Update common.jl's datamat to match your mcmc's chain's time span if it doesn't match already
## Load libraries and data

In [ ]:
# Activate the environment
#using Pkg
#Pkg.activate("");
using CSV, DataFrames, Plots, FileIO
pyplot();

# Load Bubar function library
include("common.jl"); include("odesolver.jl"); include("gibbs.jl");

# Load data
idbeg = 1; idend = 1;
M_mcmc,M_err = mergeoutputs(idbeg,idend);
nsmp = length(M_err); @assert nsmp == size(M_mcmc)[2] "Number of samples not consistent";
println("$nsmp samples taken ...");

## Declare Notebook Options

In [ ]:
#* Changed
# Declare whether a change point analysis was performed
flagΔpt = true;

# Declare final day data was fit to
#  (Notebook assumes data() outputs tspan[2] for gibbs trajectories)
dayfit = Date("2021-08-16");

# Trace plots shown every nstep
nstep = 1;

# Specify range of middle quantiles for MCMC posteriors w/wout σ-errors
qthresh = .95;
σthresh = .95;

## Analyze best fit

### Prepare to run simulation

In [ ]:
#* Changed
# Find the best fit
idbest = findfirst(M_err .== minimum(M_err));
sheet = data(); T1,T2 = sheet.tspan;
mydat,myaux = csvdat(M_mcmc[:,idbest],sheet.csv_vac,sheet.csv_odh);
sheet = data(mydat); sheet.tspan[:] = [T1,T2];
mydep = depmat(mydat,myaux); frc_M = vaxld(); # data used in sim

In [ ]:
# Load ODH data
fname = sheet.csv_odh; dfodh = CSV.read(fname,DataFrame);
cols = ["0-9","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80+"];

# Declare initial and final days for trajectories
dates = dfodh[!,:time]; # Vector of dates
day0 = Date("2020-01-01");
dayi = day0 + Day(sheet.tspan[1]);
daym = day0 + Day(sheet.tspan[2]);

In [ ]:
#* Changed
dayodh = dates[end];
dayf = maximum([dayodh,daym]);

ti = getfield(dayi-day0,:value); tf = getfield(dayf-day0,:value); tm = getfield(daym-day0,:value); 
tfit = getfield(dayfit-day0,:value); todh = getfield(dayodh-day0,:value)
plot(dfodh[!,:time],dfodh[!,:daily_confirm],labels="",title="ODH Reported Daily Infections")
vline!([dayi,dayfit],labels="")
savefig("odh.pdf")

#### Used to simulate the best fit to the last ODH day, so can compare after the fit to data

In [ ]:
# Comment out if don't want to simulate to last ODH day
mydat[:tspan] = [Float64(ti),Float64(tf)];
sheet = data(mydat); mydep = depmat(mydat,myaux); 
frc_M = (isempty(sheet.csv_vac)) ? [0. 0.;1. 0.] : vaxld(); # data used in sim

### Run the simulation

In [ ]:
# Run the simulation
tpts,ypts = odesolver(sheet,mydep,frc_M);

# Aggregate ypts across unvax, vax, and unwilling for comparing to ODH
Ypts = reshape(ypts,(27,Int64(length(ypts)/(27*length(tpts))),length(tpts)));
U = Ypts[1:9,:,:]; V = Ypts[10:18,:,:]; X = Ypts[19:27,:,:];
Ypts = U + V + X; # Ages x SEIR x Time

# Find daily new infections
#  Use ∫fdx ≈ ∑ᵢfᵢΔxᵢ = L*∑ᵢfᵢ/n
Etot = Ypts[:,2,:];
Zpts = Matrix{Float64}(undef,9,tf-ti);
for i=ti:(tf-1)
    for j=1:9
        val1 = myinterp(tpts,Etot[j,:],Float64(i));
        val2 = myinterp(tpts,Etot[j,:],Float64(i+1));
        
        Zpts[j,i-(ti-1)] = (1/sheet.d_E)*(val1+val2)/2;
    end
end

#  Scale by reporting factor
Zpts *= 1/myaux[:rptλ]*1/myaux[:rptλE];

### Plot a Markov error distribution
#### No moving average

In [ ]:
#* Change
# Noise = ODH-Model
Noise = hcat(dfodh[!,"0-9"][ti:todh-1] - Zpts[1,1:todh-ti],
         dfodh[!,"10-19"][ti:todh-1] - Zpts[2,1:todh-ti],
         dfodh[!,"20-29"][ti:todh-1] - Zpts[3,1:todh-ti],
         dfodh[!,"30-39"][ti:todh-1] - Zpts[4,1:todh-ti],
         dfodh[!,"40-49"][ti:todh-1] - Zpts[5,1:todh-ti],
         dfodh[!,"50-59"][ti:todh-1] - Zpts[6,1:todh-ti],
         dfodh[!,"60-69"][ti:todh-1] - Zpts[7,1:todh-ti],
         dfodh[!,"70-79"][ti:todh-1] - Zpts[8,1:todh-ti],
         dfodh[!,"80+"][ti:todh-1] - Zpts[9,1:todh-ti]);
Noiseᵗʳ = copy(transpose(Noise));

# Array of Present and Future errors
Mkverr = [Noiseᵗʳ[:,1:end-1][:] Noiseᵗʳ[:,2:end][:]]

# Plot Markov error distribution
histogram2d(Mkverr[:,1],Mkverr[:,2],xlabel="Present Noise",ylabel="Future Noise",title="Noise Distribution about Best Fit")
savefig("mkverr.pdf")

#### Moving average

In [ ]:
#*Change
# Noise = ODH-Model
Noise = hcat(mymvavg(dfodh[!,"0-9"][ti:todh-1]) - Zpts[1,1:todh-ti],
         mymvavg(dfodh[!,"10-19"][ti:todh-1]) - Zpts[2,1:todh-ti],
         mymvavg(dfodh[!,"20-29"][ti:todh-1]) - Zpts[3,1:todh-ti],
         mymvavg(dfodh[!,"30-39"][ti:todh-1]) - Zpts[4,1:todh-ti],
         mymvavg(dfodh[!,"40-49"][ti:todh-1]) - Zpts[5,1:todh-ti],
         mymvavg(dfodh[!,"50-59"][ti:todh-1]) - Zpts[6,1:todh-ti],
         mymvavg(dfodh[!,"60-69"][ti:todh-1]) - Zpts[7,1:todh-ti],
         mymvavg(dfodh[!,"70-79"][ti:todh-1]) - Zpts[8,1:todh-ti],
         mymvavg(dfodh[!,"80+"][ti:todh-1]) - Zpts[9,1:todh-ti]);
Noiseᵗʳ = copy(transpose(Noise));

# Array of Present and Future errors
Mkverr = [Noiseᵗʳ[:,1:end-1][:] Noiseᵗʳ[:,2:end][:]]

# Plot Markov error distribution
histogram2d(Mkverr[:,1],Mkverr[:,2],xlabel="Present Noise",ylabel="Future Noise",title="Noise Distribution about Best Fit (mvavg)")
savefig("mkverr_mvavg.pdf")

### Plot the solution with ODH data
Immediate next cell computes quantiles of MCMC trajectories without their standard deviation. Switch to markdown and comment out the plot! lines for quantiles in the 2nd next cell if not desired.

In [ ]:
# Load trajectories on all samples for MCMC quantiles
using Statistics
Mtrajs = 0;
qttaxis=0;
for i=idbeg:idend
    df = CSV.read("Run"*string(i)*"/GibbsTrajs.csv",DataFrame,header=false);
    Mtrajs = (i == idbeg) ? convert(Matrix,df[:,2:end]) : [Mtrajs convert(Matrix,df[:,2:end])];
    qttaxis = (i == idend) ? df[!,1] : 0;
end
Mᵗʳ = copy(transpose(Mtrajs)); # copy makes it genuine matrix
dwnsmp = Int64(length(qttaxis)/9);
# Compute the quantiles
Mquants = Matrix{Float64}(undef,9*dwnsmp,2);
for i=1:9*dwnsmp
    Mquants[i,:] = [quantile!(Mᵗʳ[:,i],(1-qthresh)/2) quantile!(Mᵗʳ[:,i],1-(1-qthresh)/2)];
end
quantlow = reshape(Mquants[:,1],dwnsmp,9); quanthigh = reshape(Mquants[:,2],dwnsmp,9);
mycolor = :red; # Color for MCMC bands

In [ ]:
# Compute the error bands which include ind incr error
σs = M_mcmc[177,:];
avgσ = sum(σs)/length(σs); 
confσ = quantile!(σs,σthresh);
Δσs = flagΔpt ? M_mcmc[187,:] : [NaN];
confΔσ = flagΔpt ? quantile!(Δσs,σthresh) : NaN;
days = dayi-day0;

σMᵗʳ_high = deepcopy(Mᵗʳ); # each row is trajectory, cols are 9*dwnsmp samples of a traj
σMᵗʳ_low = deepcopy(Mᵗʳ);
ntrajs = size(Mᵗʳ)[1];

# Loop over all mcmc smp's, age groups, days and include their own errors
gen = [1,1,0];
varnow = 0.; ptid = Inf; Δσnow = 0.; # declare for local scope issues
for k=1:ntrajs*9*dwnsmp
    # cycle generator: 
    #  1st comp is MCMC smp, 2nd comp is age, 3rd comp is time
    if gen[3] != dwnsmp
        gen[3] += 1;
    elseif gen[2] != 9
        gen[2] += 1; gen[3] = 1;
    else
        gen[1] += 1; gen[2] = 1; gen[3] = 1;
    end
    
    smp = gen[1]; aid = gen[2]; tid = gen[3];
    col = (aid-1)*dwnsmp+tid;
    
    # Update the var and Δpt info as needed
    if tid == 1
        # Reset var if new age group
        varnow = 0.;
        
        # Find new Δpt value if new sample
        if flagΔpt&&(aid == 1)
            Δptbest = M_mcmc[182,smp]; Δσnow = M_mcmc[187,smp];
            ptid = floor(Δptbest - days.value); ptid = (ptid > length(quantlow[:,1])-1) ? length(quantlow[:,1])-1 : ptid;
        end
    end
    
    # Determine the appropriate std dev across change point
    σnow = M_mcmc[177,smp];
    if !flagΔpt
        varnow += σnow^2; #does var need to start at 0? now it starts at not 0
    else
        varnow = (tid <= ptid) ? varnow + σnow^2 : varnow + Δσnow^2; 
    end
    
    # Compute errors
    σMᵗʳ_high[smp,col] += 2*√(varnow);
    σMᵗʳ_low[smp,col]  -= 2*√(varnow);
    σMᵗʳ_low[smp,col]  = maximum([σMᵗʳ_low[smp,col],0.]);
end
    
# Compute the σthresh quantiles for these
σMquants = Matrix{Float64}(undef,9*dwnsmp,2);
for i=1:9*dwnsmp
    σMquants[i,:] = [quantile!(σMᵗʳ_low[:,i],(1-qthresh)/2) quantile!(σMᵗʳ_high[:,i],1-(1-qthresh)/2)];
end
σquantlow = reshape(σMquants[:,1],dwnsmp,9); σquanthigh = reshape(σMquants[:,2],dwnsmp,9);
myσcolor = :blue; # Color for σMCMC bands

In [ ]:
#* Change
# Plot the data
ymax = maximum(convert(Matrix,dfodh[ti:todh,2:end-2])) + 10;
p1 = plot(ti:todh,dfodh[!,"0-9"][ti:todh],labels="ODH",title="0-9"); plot!(ti:todh,mymvavg(dfodh[!,"0-9"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[1,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); 
quantmean = .5*quantlow[:,1] + .5*quanthigh[:,1]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,1],quanthigh[:,1]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,1],σquanthigh[:,1]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p2 = plot(ti:todh,dfodh[!,"10-19"][ti:todh],labels="ODH",title="10-19"); plot!(ti:todh,mymvavg(dfodh[!,"10-19"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[2,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,2] + .5*quanthigh[:,2]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,2],quanthigh[:,2]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,2],σquanthigh[:,2]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p3 = plot(ti:todh,dfodh[!,"20-29"][ti:todh],labels="ODH",title="20-29"); plot!(ti:todh,mymvavg(dfodh[!,"20-29"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[3,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,3] + .5*quanthigh[:,3]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,3],quanthigh[:,3]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,3],σquanthigh[:,3]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p4 = plot(ti:todh,dfodh[!,"30-39"][ti:todh],labels="ODH",title="30-39"); plot!(ti:todh,mymvavg(dfodh[!,"30-39"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[4,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,4] + .5*quanthigh[:,4]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,4],quanthigh[:,4]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,4],σquanthigh[:,4]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p5 = plot(ti:todh,dfodh[!,"40-49"][ti:todh],labels="ODH",title="40-49"); plot!(ti:todh,mymvavg(dfodh[!,"40-49"][ti:todh]),labels="ODH Smooth");# plot!(ti:(tf-1),Zpts[5,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,5] + .5*quanthigh[:,5]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,5],quanthigh[:,5]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,5],σquanthigh[:,5]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p6 = plot(ti:todh,dfodh[!,"50-59"][ti:todh],labels="ODH",title="50-59"); plot!(ti:todh,mymvavg(dfodh[!,"50-59"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[6,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,6] + .5*quanthigh[:,6]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,6],quanthigh[:,6]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,6],σquanthigh[:,6]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p7 = plot(ti:todh,dfodh[!,"60-69"][ti:todh],labels="ODH",title="60-69"); plot!(ti:todh,mymvavg(dfodh[!,"60-69"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[7,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,7] + .5*quanthigh[:,7]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,7],quanthigh[:,7]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tm],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,7],σquanthigh[:,7]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p8 = plot(ti:todh,dfodh[!,"70-79"][ti:todh],labels="ODH",title="70-79"); plot!(ti:todh,mymvavg(dfodh[!,"70-79"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[8,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,8] + .5*quanthigh[:,8]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,8],quanthigh[:,8]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,8],σquanthigh[:,8]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

p9 = plot(ti:todh,dfodh[!,"80+"][ti:todh],labels="ODH",title="80+"); plot!(ti:todh,mymvavg(dfodh[!,"80+"][ti:todh]),labels="ODH Smooth"); #plot!(ti:(tf-1),Zpts[9,:],labels="Model",ribbon=2*myaux[:bayσ],fillalpha=.3,ylims=(0.,ymax)); vline!([tm],labels="");
quantmean = .5*quantlow[:,9] + .5*quanthigh[:,9]
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-quantlow[:,9],quanthigh[:,9]-quantmean),ylims=(0.,ymax), labels="MCMC Mean", fillalpha=.2, color=mycolor);vline!([tfit],labels="");
plot!(qttaxis[1:dwnsmp], quantmean, ribbon=(quantmean-σquantlow[:,9],σquanthigh[:,9]-quantmean),ylims=(0.,ymax), labels="Inc Error", fillalpha=.2, color=myσcolor);

println("Dates run from $dayi to $dayf while simulated up to $daym")
lay = @layout [a b c; d e f; g h i];
plot(p1,p2,p3,p4,p5,p6,p7,p8,p9, layout=lay, size = (1000,650))
savefig("prediction.pdf")

In [ ]:
ENV["COLUMNS"] = 1000; ENV["ROWS"] = 1000;
println("Best parameters:")
df = DataFrame(Dict{String,Float64}("L1 Err"=>M_err[idbest],"r0"=>M_mcmc[12,idbest],
                                    "α"=>M_mcmc[31,idbest],"ω"=>M_mcmc[32,idbest],
                                    "rptλ"=>M_mcmc[176,idbest],"bayσ"=>M_mcmc[177,idbest],
                                    "vι0"=>M_mcmc[179,idbest],
                                    "rptλE"=>M_mcmc[180,idbest],"rptλI"=>M_mcmc[181,idbest],
                                    "Δpt"=>M_mcmc[182,idbest],"Δr0"=>M_mcmc[183,idbest],
                                    "Δα"=>M_mcmc[184,idbest],"Δω"=>M_mcmc[185,idbest],
                                    "Δrptλ"=>M_mcmc[186,idbest],"Δbayσ"=>M_mcmc[187,idbest]))

## Collect marginal distributions

In [ ]:
p1 = histogram(M_mcmc[12,:],normalize=:probability, labels="", title="r0"); #r0
p2 = histogram(M_mcmc[31,:],normalize=:probability, labels="", title="α"); #α
p3 = histogram(M_mcmc[32,:],normalize=:probability, labels="", title = "ω"); #ω
p4 = histogram(M_mcmc[176,:],normalize=:probability, labels="", title="rptλ"); #rptλ
p5 = histogram(M_mcmc[177,:],normalize=:probability, labels="", title="bayσ"); #bayσ
#p6 = histogram(M_err,normalize=:probability, labels="", title="MSE"); #L1 Err
p7 = histogram(M_mcmc[179,:],normalize=:probability,labels="",title="vι0"); #vι0
#p8 = histogram(M_mcmc[180,:],normalize=:probability,labels="",title="rptλE"); #rptλE
#p9 = histogram(M_mcmc[181,:],normalize=:probability,labels="",title="rptλI"); #rptλI
#p10 = histogram(M_mcmc[182,:],normalize=:probability,labels="",title="Δt"); #Δt
#p11 = histogram(M_mcmc[183,:],normalize=:probability,labels="",title="r0λ"); #r0λ

lay = @layout [a b;c d; e f];
plot(p2,p3,p1,p4,p5,p7, layout=lay, size=(850,425))
savefig("marginals.pdf")

In [ ]:
# Change point marginals
if flagΔpt
    p1 = histogram(M_mcmc[183,:],normalize=:probability, labels="", title="Δr0"); #r0
    p2 = histogram(M_mcmc[184,:],normalize=:probability, labels="", title="Δα"); #α
    p3 = histogram(M_mcmc[185,:],normalize=:probability, labels="", title = "Δω"); #ω
    p4 = histogram(M_mcmc[186,:],normalize=:probability, labels="", title="Δrptλ"); #rptλ
    p5 = histogram(M_mcmc[187,:],normalize=:probability, labels="", title="Δbayσ"); #bayσ
    #  If Δpt was Infty substitute dummy plot for histogram
    p7 = (minimum(M_mcmc[182,:]) != Inf) ? histogram(M_mcmc[182,:],normalize=:probability,labels="",title="Δpt") : plot([0.,0.],[0.,0.],labels="",title="Did not run Δpt") ; #Δpt
    
    lay = @layout [g h; a b; c d];
    plot(p7,p5,p2,p3,p1,p4, layout=lay, size=(850,425))
    savefig("marginals_dltpt.pdf")
end

## Trace Plots

In [ ]:
q1 = plot(M_mcmc[12,1:nstep:end],labels="", title="r0"); #r0
q2 = plot(M_mcmc[31,1:nstep:end],labels="", title="α"); #α
q3 = plot(M_mcmc[32,1:nstep:end],labels="", title = "ω"); #ω
q4 = plot(M_mcmc[176,1:nstep:end],labels="", title="rptλ"); #rptλ
q5 = plot(M_mcmc[177,1:nstep:end],labels="", title="bayσ"); #bayσ
#q6 = plot(M_err,labels="",title="MSE")
q7 = plot(M_mcmc[179,1:nstep:end],labels="",title="vι0"); #vι0
#q8 = plot(M_mcmc[180,:],labels="",title="rptλE"); #rptλE
#q9 = plot(M_mcmc[181,:],labels="",title="rptλI"); #rptλI
#q10 = plot(M_mcmc[182,:],labels="",title="Δt"); #Δt
#q11 = plot(M_mcmc[183,:],labels="",title="r0λ"); #r0λ

lay = @layout [a b;c d; e f];
plot(q2,q3,q1,q4,q5,q7, layout=lay, size=(850,425))
savefig("trace.pdf")

In [ ]:
# Change point trace plots
if flagΔpt
    q1 = plot(M_mcmc[183,1:nstep:end],labels="", title="Δr0"); #r0
    q2 = plot(M_mcmc[184,1:nstep:end],labels="", title="Δα"); #α
    q3 = plot(M_mcmc[185,1:nstep:end],labels="", title = "Δω"); #ω
    q4 = plot(M_mcmc[186,1:nstep:end],labels="", title="Δrptλ"); #rptλ
    q5 = plot(M_mcmc[187,1:nstep:end],labels="", title="Δbayσ"); #rptλ
    q7 = plot(M_mcmc[182,1:nstep:end],labels="",title="Δpt"); #Δpt
    
    lay = @layout [g h; a b; c d];
    plot(q7,q5,q2,q3,q1,q4, layout=lay, size=(850,425))
    savefig("trace_dltpt.pdf")
end